To start, call the contents of my package, ignore the warnings or errors, this is because somehow we are redefining something by calling to packages that have the same function

In [1]:
using Revise
using FractionalKrylovMC

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


We create a random problem

In [2]:
# USAGE: problem = create_random_problem(α::ExponentInTime,γ::ExponentInSpace,k::SizeOfVector)
α = 0.9 # using derivative in time ^C_0 D_t^α
γ = 0.8 # using A^γ as the linear operator
# perhaps lower this!
k = 300 # size(u0) = k; size(A) = (k,k)
problem = create_random_problem(α,γ,k,MittagLefflerRand())

MittagLefflerProblem{Float64}(300, [0.4824394867401651 -0.00722171210449179 … -0.01090027785075779 -0.0008844858747993466; -0.0072217121044923 0.4929662395824477 … -0.025366388413776246 0.03872790641056803; … ; -0.01090027785075802 -0.025366388413776142 … 0.5189829017111584 0.004329445535263789; -0.0008844858747996762 0.03872790641056792 … 0.004329445535263793 0.5101327511941727], [0.4824394867401651 -0.00722171210449179 … -0.01090027785075779 -0.0008844858747993466; -0.0072217121044923 0.4929662395824477 … -0.025366388413776246 0.03872790641056803; … ; -0.01090027785075802 -0.025366388413776142 … 0.5189829017111584 0.004329445535263789; -0.0008844858747996762 0.03872790641056792 … 0.004329445535263793 0.5101327511941727], [0.5426309710050707, 0.2278391218169208, 0.8331563058665571, 0.016776096962153986, 0.4087816659958188, 0.1119005507365094, 0.8768419091319543, 0.5447832195946511, 0.553502531273299, 0.203565234460385  …  0.4421874479965595, 0.17090405442454826, 0.07221719128180293, 0

problem has following fields

In [ ]:
# relevant parameters of the problem
problem.A # matrix
problem.u0 # initial condition
problem.t # time
# we want to compute
# mittleff(α,-A^γ*t^α)*u0

the way I create the problem, I already compute the exact solution

In [ ]:
problem.uT_spectral

we can call the Monte-Carlo solver, which computes the exponential matrix by computing $\exp(-At)$ for all random times $t$ and does not project to any subspace

In [3]:
nsims = Int(1e5)
cutoff = 1e6 # ignoring all times beyond cutoff
MCSol = solve(problem,nsims,cutoff,MCSolver())

FractionalKrylovMC.MittagLefflerMCSolution{Float64}([0.36714449081163797, 0.16054479224034057, 0.7087889003337605, 0.09496610322228217, 0.43542200230018957, 0.21124649674315274, 0.7212987105816101, 0.41959887371719296, 0.49372912924625056, 0.23225349751277044  …  0.3223293753086675, 0.12431296863590438, 0.1019476738761953, 0.8443861179070706, 0.5717494720550758, 0.693510869540203, 0.4737610420673497, 0.11057805280176222, 0.2551225505365903, 0.6557319443299396], [9.018043827341159e-8, 7.624961154888441e-7, 2.642492874926617e-6, 5.366256055021698e-6, 1.5090989019786715e-5, 1.5587282840024563e-5, 1.918102561295016e-5, 2.2303104506077277e-5, 2.3283488152344732e-5, 2.3301475717808258e-5  …  2534.079598013102, 3122.100360232664, 6228.928226374932, 8750.827586839128, 9657.88809282001, 10588.808157937881, 13361.092769772798, 16932.696921599985, 41524.36342069755, 49573.1272526413], 100000, 1.0e6)

Check relative error and random entries to see whether it is correct, checking with confidence ranges will be done in the future

In [4]:
using Random
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCSol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.0008520554223556011
problem.uT_spectral[entries_to_show] = [0.12414436853620948, 0.6275463597562488, 0.7879545326946402, 0.8035879390711249, 0.24484002331160093]
MCSol.uT[entries_to_show] = [0.12431296863590438, 0.6281522186445314, 0.788744477496306, 0.8040513603493401, 0.24477503862721262]


can also use Krylov!

In [5]:
KrySubspaceDim = 5
MCKrySol = solve(problem,nsims,cutoff,KrySubspaceDim,MCSolverExpokit())

MittagLefflerMCSolution{Float64}([0.11636792768055922, 0.03590590232145222, 0.09095050889481589, 0.5877801084548403, 0.3312206673144261, 0.5808196195255234, 0.3774191449106153, 0.12705843474498177, 0.2987390918440006, 0.399895148964329  …  0.25722577350422654, 0.07298003390713695, 0.018049700787571336, 0.08802300657120808, 0.7026595843706074, 0.19096146424429045, 0.7175306864704117, 0.22248998464868872, 0.36509778629551143, 0.29578879059167656], [2.2923520499773145e-6, 3.799145958859502e-6, 1.5132093552727634e-5, 1.6060090715816866e-5, 1.7576099950020373e-5, 1.9807230255466673e-5, 2.11699004618462e-5, 2.614284110197647e-5, 2.758549532932784e-5, 2.8661746962706307e-5  …  23523.68249820971, 24975.751542023347, 26856.316512089554, 27225.547935359755, 28895.5865803514, 31755.969661765645, 38945.8723638231, 48549.76383027059, 55477.94722272533, 84481.47328431379], 100000, 1.0e6)

In [6]:
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCKrySol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.04405151545783162
problem.uT_spectral[entries_to_show] = [0.32903371631807354, 0.466519351346999, 0.39266344200827225, 0.42292447860218824, 0.3700733660968992]
MCKrySol.uT[entries_to_show] = [0.3435055952725601, 0.48435786832338157, 0.40558008261410644, 0.4253704811065888, 0.3735129089014563]
